## Install libraries

In [0]:
!pip install -qU openai
!pip install -qU llama-index
!pip install -qU pydantic
!pip install -qU sentence-transformers
!pip install -qU llama-index-llms-azure-openai
!pip install -qU llama-index-embeddings-azure-openai
!pip install plotly

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packag

## Imports

In [0]:
import os
import warnings
warnings.filterwarnings("ignore")
import openai


import json
import re

import matplotlib.pyplot as plt
import plotly.express as px

from datasets import load_dataset
from llama_index.core import (
    Document,
    VectorStoreIndex,
    Settings
)

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

## Configure Azure OpenAI

In [0]:
endpoint = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/" 
model_name = "gpt-5-nano" 
deployment = "gpt-5-nano" 

subscription_key = "" 
api_version = "2024-12-01-preview" 

Settings.llm = AzureOpenAI( 
    api_version=api_version, 
    api_key=subscription_key, 
    azure_endpoint=endpoint,
    engine=deployment,
    temperature=1.0
) 

Settings.embed_model = AzureOpenAIEmbedding(
  model=embed_model_name,
  deployment_name=embed_model_deployment_name,
  api_key=openai.api_key,
  azure_endpoint=openai.api_base
  )

## Load Dataset

**Dataset: _Financial QA 10K_**

**Source:** [Hugging Face](https://huggingface.co/datasets/virattt/financial-qa-10K)

In [0]:
ds = load_dataset("virattt/financial-qa-10K", split="train")

Each row contains:

* Question
* Answer
* Context - text paragraph that contains the information required to answer the question; core of our RAG system.
* Ticker - symbol of the company the context belongs to.
* Filing - filing identifier (which year the data came from).

In [0]:
ds[0]

{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'answer': 'NVIDIA initially focused on PC graphics.',
 'context': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'ticker': 'NVDA',
 'filing': '2023_10K'}

In [0]:
ds_small = ds.select(range(300))

## Convert contexts into LlamaIndex Documents

In [0]:
documents = [Document(text=row["context"]) for row in ds_small]

## Build a Vector Index & Create a Query Engine

* LlamaIndex splits documents into chunks, embeds each of them and then stores embeddings into a vector index
* Then the vector index is wrapped into a query engine

In [0]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

[Trace(request_id=tr-1e9026344b1041818cebaf9336d5ad5e), Trace(request_id=tr-1237b3a61cfa414b85dd31552507d5cc), Trace(request_id=tr-e4408753bbd446468651610801f374eb), Trace(request_id=tr-eee8f5247e2b49808a51e8234562b9b0), Trace(request_id=tr-ff39da8b56b34c1db8d20ebc893d6b54), Trace(request_id=tr-5ef990bbaf17483f93557f11db611dcc), Trace(request_id=tr-0858fd857ab94647aac1ee8709f8a313), Trace(request_id=tr-b3c7030e1c9a429aa6c3a39d1c2d37b4), Trace(request_id=tr-3564efcaa53542919d1bfd548eba37f1), Trace(request_id=tr-59459b8329164ef88bcda28335cc12b2)]

## Test the simple RAG

In [0]:
def finance_agent(question):
  return query_engine.query(question).response

In [0]:
finance_agent("What risks does the company mention")

'- Global macroeconomic and industry risks: dependence on international markets, with a majority of sales outside the U.S., and a large, global supply chain. Adverse conditions can reduce demand and affect suppliers, contract manufacturers, logistics providers, distributors, carriers, and developers.\n\n- Economic factors impacting demand: slow growth or recession, high unemployment, inflation, tighter credit, higher interest rates, currency fluctuations, and shifts in fiscal/monetary policy that affect consumer confidence and spending.\n\n- Financial/credit risks: increased credit and collectibility risk on trade receivables, potential counterparty defaults, financial instability of financial institutions, limitations on issuing new debt, reduced liquidity, and declines in the value of financial instruments.\n\n- Political and disruption risks: events such as political tensions, trade disputes, war, terrorism, natural disasters, public health issues, industrial accidents, and other ev

[Trace(request_id=tr-a8913749fb2a4a42881bd9c377a0b1ee), Trace(request_id=tr-a6bc189b5fa44ecc9bba156d902baeec)]

In [0]:
row = ds_small[42]
true_answer = row["answer"]
question = row["question"]

In [0]:
predicted_answer = finance_agent(question)

[Trace(request_id=tr-5e8b9948cf10450b940db85074ae2686), Trace(request_id=tr-7f7e90960e044dc2ad6aaee0598c9fb5)]

In [0]:
print(question)
print(true_answer)
print(predicted_answer)

What was the overall turnover rate at the company in fiscal year 2023?
The overall turnover rate at the company in fiscal year 2023 was 5.3%.
5.3%


## Building an agent

In [0]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import QueryEngineTool, FunctionTool
import plotly.express as px
import json

## Tools

### 1. RAG Tool

In [0]:
financial_rag = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="financial_rag",
    description="Retrieve relevant financial context from the dataset."
)

### 2. Visualization tool

We will get from the LLM a JSON in this format: 

`{`

`  "chart_type": "...",`

`  "labels": [...],`

`  "values": [...]`

`}`

The tool will then clean the numbers and display the suitable Plotly chart.

In [0]:
def visualize_numbers(chart_type: str, labels: list, values: list):
    """Plot numeric data with Plotly."""
    clean_values = []
    for v in values:
        try:
            clean_values.append(float(str(v).replace("%", "").replace(",", "")))
        except:
            pass

    if len(labels) != len(clean_values):
        labels = [f"Value {i+1}" for i in range(len(clean_values))]

    # choose chart
    if chart_type == "comparison":
        fig = px.bar(x=labels[:2], y=clean_values[:2], title="Comparison Chart")
    elif chart_type == "pie":
        fig = px.pie(names=labels, values=clean_values, title="Pie Chart")
    elif chart_type == "line":
        fig = px.line(x=labels, y=clean_values, markers=True, title="Line Chart")
    else:
        fig = px.bar(x=labels, y=clean_values, title="Bar Chart")

    fig.show()
    return "Visualization done."

In [0]:
visualize_tool = FunctionTool.from_defaults(
    name="visualize_numbers",
    fn=visualize_numbers,
    description="Visualize numeric values using Plotly."
)

## Multi-Tool Agent

In [0]:
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[financial_rag, visualize_tool],
    llm=Settings.llm,
    verbose=True,
    system_prompt="""
You are a financial analysis agent.

### YOUR REQUIRED WORKFLOW:
1. Use the `financial_rag` tool to retrieve context.
2. Extract ALL numeric values relevant to the user question.
3. Decide the BEST chart type: bar, comparison, line, or pie.
4. Return ONLY JSON (no text) in this format:
{
  "chart_type": "...",
  "labels": [...],
  "values": [...]
}
5. Then IMMEDIATELY call `visualize_numbers` with that JSON.
6. When done, return the final textual answer.

NEVER output plain text before the JSON.
NEVER skip visualization if numbers exist.
"""
)

In [0]:
response = await agent.run(
    "How did the revenue from automotive regulatory credits change in 2023 compared to 2022?"
)

print(response)

- 2022 automotive regulatory credits revenue: $1,400 million
- 2023 automotive regulatory credits revenue: $1,414 million
- Change: increase of $14 million year-over-year (from 2022 to 2023)




[Trace(request_id=tr-91830de5435b49a081c397ed2f9a5be3), Trace(request_id=tr-90c9a7e34b38419e820257fc9b109a2f), Trace(request_id=tr-e038c8e3b1ad4cac9920dcf42dfc9a4f), Trace(request_id=tr-3bcab758d6234eb49eebb6b4eca7343c), Trace(request_id=tr-0a4f81a7de014e1c91399cacbf1bacc7)]

In [0]:
response = await agent.run(
    "What percentage of the global workforce was female at the end of fiscal year 2023?"
)

print(response)

{
  "chart_type": "pie",
  "labels": ["Female", "Male"],
  "values": [19, 81]
}

Visualization done.


[Trace(request_id=tr-2873af39a1964c34886261f6ad5082de), Trace(request_id=tr-204360fb160141e68a72441d143f003a), Trace(request_id=tr-0cb7fc2bf9ec4503acf206a182a08549), Trace(request_id=tr-3b7132ea539a4ef3ac24b57d319b6282), Trace(request_id=tr-0a327cfcb4854ef6ad8c7173497a96b4)]

In [0]:
response = await agent.run(
    "What is the percentage of Black or African American and Hispanic or Latino employees in the workforce in the United States?"
)

print(response)

The combined percentage of Black or African American and Hispanic or Latino employees in the U.S. workforce is 6%.


[Trace(request_id=tr-73d227f61efa4c6ea7d7e37f1f202b04), Trace(request_id=tr-7c2baf21ee2542eaa1cae8c32b0327ad), Trace(request_id=tr-176a7c8f210c4b998e420ee6a2d79201), Trace(request_id=tr-5f75e8fcdcae4e5695548b518fa241f5), Trace(request_id=tr-58ef3d25298c421e80b4ef42f1346db2)]